# Repàs pandas

## Lectura, tipus de dades i duplicats

In [1]:
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

Treballarem amb una llista de noms extrets de https://www.idescat.cat/noms/

In [2]:
df = pd.read_csv("data/noms.csv", sep=";", skiprows=7)
df

,Pos.,Sexe,Nom,Rànquing. Freqüència,Rànquing. ‰
0,1,H,ANTONIO,89.957,"11,54"
1,2,D,MARIA/MARÍA,88.985,"11,42"
2,3,H,JOSÉ,84.830,"10,89"
3,4,D,MONTSERRAT,75.819,"9,73"
4,5,H,JORDI,72.247,"9,27"
...,...,...,...,...,...
59129,46701,H,ZUPING,4.000,"0,00"
59130,46701,D,ZURI,4.000,"0,00"
59131,46701,H,ZVONIMIR,4.000,"0,00"
59132,46701,H,ZVONKO,4.000,"0,00"


In [13]:
df.sort_values(by="Rànquing. Freqüència").Nom[:100]

Object `sample` not found.


In [17]:
df.sort_values(by="Rànquing. Freqüència").Nom[:100].sample(50).to_list()

['SEBASTIÀ',
 'MAURO',
 'MARIA TRINIDAD/MARÍA TRINIDAD',
 'FLORENCIO',
 'VIOLETA',
 'JAMILA',
 'MARWA',
 'AMADOR',
 'JORDINA',
 'VICTORIANO',
 'FLORÈNCIA/FLORENCIA',
 'ELNA',
 'MARYAM',
 'SOUAD',
 'FATOUMATA',
 'FIDEL',
 'MARISOL',
 'ITZIAR',
 'JOSEPA',
 'SIRA',
 'DEREK',
 'JOFRE',
 'CATERINA',
 'IVETTE',
 'MIREYA',
 'MARCELO',
 'AZIZ',
 'FLORA',
 'AMADEO',
 'NÈSTOR/NÉSTOR',
 'NABIL',
 'IMANE',
 'JUAN IGNACIO',
 'CÀNDIDA/CÁNDIDA',
 'HAMID',
 'TONI',
 'ISIDORO',
 'ERICA',
 'SAMIR',
 'CORAL',
 'JON',
 'FATIMA ZOHRA',
 'LUIS FERNANDO',
 'PEDRO ANTONIO',
 'GRETA',
 'GREGÒRIA/GREGORIA',
 'ADELAIDA',
 'EUDALD',
 'IKRAM',
 'MARIA BEGOÑA/MARÍA BEGOÑA']

Renombrem les columnes

In [ ]:
df = df.rename(columns={"Rànquing. Freqüència": "freq", "Rànquing. ‰": "permil"})

El primer que cal fer és mirar les dades a mà, per saber què contenen.

En quin format està llegint les dades pandas? (`object` ve a ser `string`)

Veiem que el rànquing de freqüència és un nombre real en comptes d'un enter,  i el rànquing en tant per mil no el reconeix com a numèric.

In [ ]:
df.info()

In [ ]:
df["freq"] *= 1000
df = df.astype({"freq": int})
df["permil"] = pd.to_numeric(df["permil"].str.replace(",", "."))
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

Si ja sabem que totes les columnes tenen el mateix format, podem directament dir-li a pandas com es marquen els decimals i els milers

In [ ]:
pd.read_csv("data/noms.csv", sep=";", skiprows=7, decimal=",", thousands=".").head()

Hi ha algun duplicat a les dades?

In [ ]:
df[df.Nom.duplicated()]

Sembla que molts, però...

In [ ]:
df[df[["Nom", "Sexe"]].duplicated()]

Era perquè eren tant per `Sexe` H com D. Realment només hi ha un duplicat

In [ ]:
df.query("Nom == 'BEGOÑA'")

## Ens falta netejar la columna `Nom`

Veiem que hi ha un nom que es llegeix com a NaN (Not a Number). Si anem a veure les dades el nom és "NA", que pandas interpreta com a NaN.
Ho podem solucionar passant l'argument `keep_default_na=False` a `pd.read_csv`

In [ ]:
df[df.Nom.isna()]

In [ ]:
df = pd.read_csv("data/noms.csv", sep=";", skiprows=7, decimal=",", thousands=".", keep_default_na=False)
df[df.Nom.isna()]

In [ ]:
df.iloc[6334]

Molts noms tenen dues o més opcions d'escriptura, amb accent o sense; amb accent obert o tancat, etc.

En podem veure la distribució i casos en particular:

In [ ]:
plt.hist(df["Nom"].str.split("/").str.len())
plt.yscale("log")

In [ ]:
df[df["Nom"].str.contains("/").fillna(False)]

Per ara optarem per quedar-nos sempre amb una sola opció. A més ho passem tot a minúscules i treiem els accents.

In [ ]:
def treure_accents(s):
    s = (
        s.replace("à", "a")
        .replace("á", "a")
        .replace("è", "e")
        .replace("é", "e")
        .replace("í", "i")
        .replace("ï", "i")
        .replace("ò", "o")
        .replace("ó", "o")
        .replace("ú", "u")
        .replace("ü", "u")
    )
    return s

In [ ]:
df["Nom"] = df["Nom"].str.split("/").str[0].str.lower().apply(treure_accents)
df.Nom

## Posant-ho tot junt:

In [ ]:
df = pd.read_csv("data/noms.csv", sep=";", skiprows=7, decimal=",", thousands=".", keep_default_na=False)
df = df.rename(columns={"Rànquing. Freqüència": "freq", "Rànquing. ‰": "permil"})
df["Nom"] = df["Nom"].str.split("/").str[0].str.lower().apply(treure_accents)
df

Podem guardar aquestes dades en un altre fitxer.

`index=False` evita guardar la primera columna (índex)

In [ ]:
df.to_csv("data/noms_net.csv", index=False)

I el podem rellegir i comprovar que tenim les mateixes dades

In [ ]:
df2 = pd.read_csv("data/noms_net.csv", keep_default_na=False)
(df == df2).all()